In [ ]:
# Install RDKit.
%%capture
!pip install rdkit-pypi
!pip install pubchempy
!pip install kora py3Dmol

In [ ]:
################# Alle Kommentare die nicht direkt gebraucht werden wurden mit Absicht nicht gelöscht, falls der Algorithmus von jemandem weiter entwickelt wird ###########################

import pubchempy as pcp 
import rdkit
from pubchempy import *
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
import kora.install.rdkit
import py3Dmol
import random

#Valenz (Wertigkeit, Anzahl möglicher Bindungen) von
#Kohlenstoff: 4
#Wasserstoff: 1
#Sauerstoff: 2

#Datenstruktur für Atom (Python-Implementation):

from typing import Dict, List

#allgemeines Atom mit Attributen
class Atom:
  idCounter: int = 0
  id: int

  # nachbarn: Dict[Atom, int] = {}
  nachbarn = None
  valenz: int
  name: str
  formel: str

  #__init__ funktioniert wie ein Konstruktur
  def __init__(self, valenz: int, name: str, formel: str):
    # Statischer Counter zur Vergabe eindeutiger IDs
    Atom.idCounter += 1
    self.id = Atom.idCounter
    self.nachbarn = {}

    self.formel = formel
    self.valenz = valenz
    self.name = name

  # Überschreiben die Standardimplementation, um Klasse als Schlüsselwert in einem Dictionary nutzbar zu machen
  # zum vergleichen ob es das gleiche Atom ist (self wird mit instanceToCompare verglichen)
  def __eq__(self, instanceToCompare):
    return hasattr(instanceToCompare, 'id') and self.id == instanceToCompare.id

  def __hash__(self):
    return hash(self.id)
  
  def getValenz(self):
    return self.valenz
  
  def addNachbar(self, nachbar, bindungsAnzahl: int):
    self.nachbarn[nachbar] = bindungsAnzahl
    nachbar.nachbarn[self] = bindungsAnzahl

  def removeNachbar(self, nachbar, bindungsAnzahl: int):
    del self.nachbarn[nachbar]
    del nachbar.nachbarn[self]

  def getAnzahlFreierBindungen(self):
    return self.valenz-sum([bindungsAnzahl for nachbar, bindungsAnzahl in self.nachbarn.items()])

#Wassserstoff, Sauerstoff und Kohlenstoff Klassen

class Wasserstoff(Atom):
  #funktioniert wie ein Konstruktur, super greift auf die Elternklasse (Atom) zu
  def __init__(self):
    super(Wasserstoff, self).__init__(1, "Wasserstoff","H")

class Sauerstoff(Atom):
  #funktioniert wie ein Konstruktur, super greift auf die Elternklasse zu
  def __init__(self):
    super(Sauerstoff, self).__init__(2, "Sauerstoff","O")

class Kohlenstoff(Atom):
  #funktioniert wie ein Konstruktur, super greift auf die Elternklasse zu
  def __init__(self):
    super(Kohlenstoff, self).__init__(4, "Kohlenstoff","C")

#ENDE der Datenstruktur für Atom.

#=========================Funktionen===========================

def choose_atom(): 
      i = random.uniform(0,1)
      if i <= 0.5:
        random_atom = atoms[2]
      if i > 0.5 and i <= 0.75:
        random_atom = atoms[1]
      if i > 0.75:
        random_atom = atoms[0]
      new_atom = random_atom()
      return new_atom

def number_of_nachbarn(dict):
      count = 0
      for key, value in dict.items():
        count +=1
      return count

def bindungen_erhöhen():
      if a.getAnzahlFreierBindungen == 0:
        Nachbarn_mit_freien_Bindungen.clear()                 #wenn Bindungen erhöht und a keine freien Bindungen mehr hat aber Nachbarn noch, dann muss array gecleart werden   
      b = random.choice(Nachbarn_mit_freien_Bindungen)
      bindungen = b.nachbarn[a] + min(a.getAnzahlFreierBindungen(),b.getAnzahlFreierBindungen())  #aktuelle Bindungen + minimum an freier Valenz von bei den Atomen (wenn Kohlenstoff 2 frei hat und Sauerstoff 1 dann kann es die Bindung nur um 1 erhöhen)
      b.nachbarn[a] = bindungen   
      a.nachbarn[b] = bindungen   #beide ändern, da sonst in beiden verschiedene Anzahl an Bindungen steht
      if (b.getAnzahlFreierBindungen == 0):                   #wenn Nachbar keine freien Bindungen mehr hat --> aus array entfernen
          Nachbarn_mit_freien_Bindungen.remove(b)

def atom_ersetzen(atom, a):
  nachbarn_von_a = []
  neues_atom = atom()

  #print(f'{a} mit {neues_atom} ersetzen')

  for Nachbar in a.nachbarn:
    nachbarn_von_a.append(Nachbar)                              #Nachbarn mit Bindungen von a speichern

  for Nachbar in nachbarn_von_a:                                #extra schleife, weil sonst Dictionary in Schleif geändert wird --> error
    #print(f'Nachbarn von zu ersetzendem Atom: {a.nachbarn}')
    Nachbar.addNachbar(neues_atom, a.nachbarn[Nachbar])
    Nachbar.removeNachbar(a, a.nachbarn[Nachbar])               #alle Nachabrn von a enternen
    #print(f'Nachbarn von zu ersetzendem Atom: {a.nachbarn}')
    #print(f'Nachbarn von neuem Atom: {neues_atom.nachbarn}')

  M.remove(a)
  M.append(neues_atom)
  nachbarn_von_a.clear()

  if len(Atome_mit_freien_Bindungen) > 0:
    Atome_mit_freien_Bindungen.remove(a)

def wasserstoff_anhängen(a):
      i = 0
      while i < a.getAnzahlFreierBindungen():
        i+=1
        h = atoms[0]
        wasserstoff_atom = h()
        M.append(wasserstoff_atom)
        a.addNachbar(wasserstoff_atom,1)
      if len(Atome_mit_freien_Bindungen) > 0:    #ansonsten versucht es a nochmal zu entfernen, wenn keine Atome mit freien Bindungen mehr da sind --> error             
        Atome_mit_freien_Bindungen.remove(a)     #wenn Nachbar keine freien Bindungen mehr hat --> aus array entfernen

#===============Funktionen für Ausgabe=============

#Adjazenzmatrix erstellen um später eigenständig SMILES Code zu berechnen
def create_adjacency_matrix():
  w, h = len(M), len(M)
  adjazenzmatrix = [[0 for x in range(w)] for y in range(h)] 

  for i in range(len(M)):
    for j in range(len(M)):
      Atom = M[j]
      try:
        adjazenzmatrix[i][j] = M[i].nachbarn[Atom]
      except: 
        adjazenzmatrix[i][j] = 0

  return adjazenzmatrix


def MolFromGraphs(node_list, adjacency_matrix):
        #Molekül mit einzelnen Formelzeichen
        for Atom in M:
          atom_formeln.append(Atom.formel)
        print(f'Molekül: {atom_formeln}')

        # create empty editable mol object
        mol = Chem.RWMol()

        # add atoms to mol and keep track of index
        node_to_idx = {}
        for i in range(len(node_list)):
            a = Chem.Atom(node_list[i])
            molIdx = mol.AddAtom(a)
            node_to_idx[i] = molIdx

        # add bonds between adjacent atoms
        for ix, row in enumerate(adjacency_matrix):
            for iy, bond in enumerate(row):

                # only traverse half the matrix
                if iy <= ix:
                    continue

                # add relevant bond type (there are many more of these)
                if bond == 0:
                    continue
                elif bond == 1:
                    bond_type = Chem.rdchem.BondType.SINGLE
                    mol.AddBond(node_to_idx[ix], node_to_idx[iy], bond_type)
                elif bond == 2:
                    bond_type = Chem.rdchem.BondType.DOUBLE
                    mol.AddBond(node_to_idx[ix], node_to_idx[iy], bond_type)
                elif bond == 3:
                    bond_type = Chem.rdchem.BondType.TRIPLE
                    mol.AddBond(node_to_idx[ix], node_to_idx[iy], bond_type)

        # Convert RWMol to Mol object
        mol = mol.GetMol()            

        return mol

#SMILES Code berechnen
def SMILES_Code(adjazenzmatrix):
  smiles = Chem.MolToSmiles(MolFromGraphs(atom_formeln, adjazenzmatrix))
  print(f'SMILES: {smiles}')
  return smiles

#Name + Formel von PubChem holen
def get_from_PubChem():
  try:
      molekül_name = pcp.get_properties('IUPACName', smiles, 'smiles')
      print(f'Name: {molekül_name}')
      molekül_formel = pcp.get_properties('MolecularFormula', smiles, 'smiles')
      print(f'Formel: {molekül_formel}')
      if molekül_name[0]['CID'] == 0:
        nicht_gefunden += 1
  except:
    print('Molekül in PubChem-Datenbank nicht gefunden')


#Ausgabe 3D Ansicht
def show(smi, style='stick'):
      mol = Chem.MolFromSmiles(smi)
      mol = Chem.AddHs(mol)
      AllChem.EmbedMolecule(mol)
      AllChem.MMFFOptimizeMolecule(mol, maxIters=200)
      mblock = Chem.MolToMolBlock(mol)
      view = py3Dmol.view(width=600, height=600)
      view.addModel(mblock, 'mol')
      view.setStyle({style:{}})
      view.zoomTo()
      view.show()

#Mermaid String wird im Algorithmus nicht mehr verwendet, aber vielleicht hilft er für den Überblick falls der Code von jemandem weiter entwickelt wird.      
def mermaid_string():
    H = 0
    O = 0
    C = 0
    #print('Mermaid String:')
    for Atom in M:
      if Atom.name == 'Wasserstoff':
        H +=1
      if Atom.name == 'Sauerstoff':
        O +=1
      if Atom.name == 'Kohlenstoff':
        C +=1
      #for Nachbar in Atom.nachbarn:
        #print(f'{Atom.id}(({Atom.name})) ---|{Atom.nachbarn[Nachbar]}| {Nachbar.id}(({Nachbar.name})) ')

#============================= Main ====================================

nicht_gefunden = 0
ungültig = 0
anzahl = 0
weiter = 0
atoms = (Wasserstoff, Sauerstoff, Kohlenstoff)

#User input
n = int(input("Wie viele Atome soll das Molekül haben?"))
durchläufe_innen = int(input("Wie viele Moleküle möchten Sie?"))

for durchgang in range(durchläufe_innen):
  durchgang = durchgang + 1
  atoms = (Wasserstoff, Sauerstoff, Kohlenstoff)


  M = []
  Molekül_mit_Namen = []
  Atome_mit_freien_Bindungen = []
  Nachbarn_mit_freien_Bindungen = []
  freieBindungen = True
  n =+ n
  Atome_mit_freien_Bindungen = []
  atom_formeln = []

  x = 0
  atom_a = choose_atom()
  M.append(atom_a)

  while len(M) < int(n):
    #schleife muss noch ersetzt werden ( nicht einfach 100x durchlaufen sondern über M gehen und alle Atome mit freien Bindungen in einem Array speichern und dann --> while array nicht leer ist)
    while (x < 100):                #gehe über gesamtes Molekül+1 (array)
      if x == 99:                         #wenn kein Atom mit freien Bindungen gefunden wurde (dann i > len(M))
          for i in range(len(M)):                     #suche Atom in Molekül mit:
            if (number_of_nachbarn(M[i].nachbarn) == 1):  #nur einen Nachbarn hat
              #print(f'test erster nachbar von {M[i]}: {next(iter(list(M[i].nachbarn.items())[0]))}') #erster nachbar (auch der einzige)
              nachbar_von_M_i = next(iter(list(M[i].nachbarn.items())[0]))
              M[i].nachbarn[nachbar_von_M_i] = M[i].nachbarn[nachbar_von_M_i] - random.randint(1,M[i].nachbarn[nachbar_von_M_i])     #entferne: Bindungen - zw(1 und Anzahl Bindungen)  
                

              if(M[i].nachbarn[nachbar_von_M_i] == 0):   #wenn keine Bindungen mehr exisitieren dann:
                #print(f'M[i]: {M[i]}')  
                #print(f'Nachbarn M[i]: {M[i].nachbarn}')
                #print(f'nachbar_von_M_i]: {nachbar_von_M_i}')
                #print(f'Nachbarn von nachbar_von_M_i: {nachbar_von_M_i.nachbarn}')                                        
                M[i].removeNachbar(nachbar_von_M_i,M[i].nachbarn[nachbar_von_M_i])    #Nachbar aus M[i] entfernen                 
                M.remove(M[i])     #Atom aus M entfernen
              i = 0 #i auf 0 damit das gesamte Molekül wieder von vorn abgesucht wird
              x = 0
              break

      random_atom_a = random.choice(M) 
      #print(f'Random Atom a aus M: {random_atom_a}') 
      x +=1                                       #suche random aus M ein Atom aus
      if random_atom_a.getAnzahlFreierBindungen() > 0: 
        x = 0                       #wenn freie Verbindung dann break aus for loop
        break

      ######if random_atom_a.getAnzahlFreierBindungen() > 0 and i

    random_atom_b = choose_atom()
    #print(f'Random erzeugtes Atom: {random_atom_b}')

    k = random.randint(1, min(random_atom_a.getAnzahlFreierBindungen(),random_atom_b.valenz)) #statt valenz Anzahl freier Bindungen bei a
    #print(f'k: {k}')
    random_atom_a.addNachbar(random_atom_b,k)
    M.append(random_atom_b)

    #print(f'M: {M}')


  ##Möglichkeit 1
  #while freieBindungen == True:
  for y in M:                                         #schauen welches Atom noch freie Verbindungen hat (nachdem M gewünschte Anzahl an Atomen hat)
    if y.getAnzahlFreierBindungen() > 0:              #wenn Atom freie Bindung hat dann in Array einfügen
      Atome_mit_freien_Bindungen.append(y)

  while freieBindungen == True:                       #solange ein Atom mit freien Bindungen existiert
    if len(Atome_mit_freien_Bindungen) > 0:           #wenn es noch ein Atom mit freien Bindungen gibt dann suche ein beliebuges davon aus
      a = random.choice(Atome_mit_freien_Bindungen)
    else:                                             #wenn Array leer ist --> keine Atome mehr mit freien Bindungen 
      break
      
    Nachbarn_mit_freien_Bindungen.clear() 

    for key, value in a.nachbarn.items():             #für jeden Nachbarn von a(Atom mit freien Bindungen)
      if key.getAnzahlFreierBindungen() > 0: 
        Nachbarn_mit_freien_Bindungen.append(key)     #array mit allen Nachbarn die freie Bindungen haben von Atom das freie Bindung hat

    #print(f' Nachbarn von {a} mit freien Bindungen: {Nachbarn_mit_freien_Bindungen}')

    if (len(Nachbarn_mit_freien_Bindungen) > 0 and a.getAnzahlFreierBindungen() > 0):        #wenn Atom Nachbarn mit freien Bindungen hat
      bindungen_erhöhen()
    else:                                             #wenn Atom keine Nachbarn mit freien Bindungen hat, aber selber noch freie Bidnungen hat
      #atome_ersetzen(a) #Durch Atome mit passender Valenz ersetzen oder H Atome auffüllen
      benötigte_valenz = a.valenz - a.getAnzahlFreierBindungen()
      if (benötigte_valenz == 2):
        atom_ersetzen(atoms[1], a)  #a durch Sauerstoff ersetzen
      elif(benötigte_valenz == 1):
        atom_ersetzen(atoms[0], a)   #a durch Wasserstoff ersetzen
      else: 
        wasserstoff_anhängen(a)

  #Notlösung um H anzuhängen:
  for x in M:
    y = 0
    freie_bindungen = x.getAnzahlFreierBindungen()
    if freie_bindungen > 0:
      while y < freie_bindungen:
        wasserstoff_anhängen(x)
        y+=1
        #print('Musste noch ein H drangehangen werden')

      

  #print('------------------------------')
  for i in M:
    #print(f'AtomID: {i.id}')
    #print(f'Name: {i.name} ({i}')
    #print(f'Valenz: {i.valenz}')      #funktioniert, gibt ID von jedem Atom in M aus
    #print(f'Bindungen übrig: {i.getAnzahlFreierBindungen()}')
    #print(f'Nachbarn: {i.nachbarn} ')
    #print('--------------')
    Molekül_mit_Namen.append(i.name)

  print(f'Molekül: {Molekül_mit_Namen}')
  #print('-------------------------------------------')

  ##Mermaid Strig
    #mermaid_string()


  ###Adjazenzmatrix erstellen um dann SMILES Code zu berechnen    
  smiles = SMILES_Code(create_adjacency_matrix())


  #mittels SMILES in PubChem Datenbank suchen um Namen+Formel zu bekommen
  get_from_PubChem()
    
  #Ausgabe der Moleküle
  print(f'3D Ansicht:')
  try:
    show(f'{smiles}')  # or 'P'
  except:
    print('Molekül ist ungültig')
    ungültig += 1
  anzahl += 1

  #print(f'2D Ansicht:' )
  #Molekül = Chem.MolFromSmiles(Chem.MolToSmiles(MolFromGraphs(atom_formeln, adjazenzmatrix)))

#Molekül


Wie viele Atome soll das Molekül haben?8
Wie viele Moleküle möchten Sie?40
Molekül: ['Kohlenstoff', 'Sauerstoff', 'Wasserstoff', 'Sauerstoff', 'Kohlenstoff', 'Wasserstoff', 'Sauerstoff', 'Wasserstoff', 'Wasserstoff']
Molekül: ['C', 'O', 'H', 'O', 'C', 'H', 'O', 'H', 'H']
SMILES: [H]OC(=O)C([H])([H])O[H]
Name: [{'CID': 757, 'IUPACName': '2-hydroxyacetic acid'}]
Formel: [{'CID': 757, 'MolecularFormula': 'C2H4O3'}]
3D Ansicht:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Molekül: ['Sauerstoff', 'Wasserstoff', 'Kohlenstoff', 'Kohlenstoff', 'Sauerstoff', 'Sauerstoff', 'Sauerstoff', 'Wasserstoff']
Molekül: ['O', 'H', 'C', 'C', 'O', 'O', 'O', 'H']
SMILES: [H]OC#COOO[H]
Name: [{'CID': 58513029, 'IUPACName': '2-(trioxidanyl)ethynol'}]
Formel: [{'CID': 58513029, 'MolecularFormula': 'C2H2O4'}]
3D Ansicht:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Molekül: ['Kohlenstoff', 'Kohlenstoff', 'Kohlenstoff', 'Sauerstoff', 'Wasserstoff', 'Kohlenstoff', 'Sauerstoff', 'Wasserstoff']
Molekül: ['C', 'C', 'C', 'O', 'H', 'C', 'O', 'H']
SMILES: [H]C(=O)C#CC([H])=O
Name: [{'CID': 11603089, 'IUPACName': 'but-2-ynedial'}]
Formel: [{'CID': 11603089, 'MolecularFormula': 'C4H2O2'}]
3D Ansicht:


...

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Molekül: ['Kohlenstoff', 'Wasserstoff', 'Sauerstoff', 'Kohlenstoff', 'Kohlenstoff', 'Kohlenstoff', 'Wasserstoff', 'Sauerstoff', 'Sauerstoff']
Molekül: ['C', 'H', 'O', 'C', 'C', 'C', 'H', 'O', 'O']
SMILES: [H]C#COC(=O)C([H])=O
Name: [{'CID': 0}]
Formel: [{'CID': 0}]
Molekül in PubChem-Datenbank nicht gefunden
3D Ansicht:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Molekül: ['Kohlenstoff', 'Sauerstoff', 'Sauerstoff', 'Sauerstoff', 'Sauerstoff', 'Wasserstoff', 'Wasserstoff', 'Sauerstoff']
Molekül: ['C', 'O', 'O', 'O', 'O', 'H', 'H', 'O']
SMILES: [H]OOOC(=O)O[H]
Name: [{'CID': 17908637, 'IUPACName': 'hydroperoxy hydrogen carbonate'}]
Formel: [{'CID': 17908637, 'MolecularFormula': 'CH2O5'}]
3D Ansicht:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol